### GIS data preparation

the solution found at https://gis.stackexchange.com/questions/157693/getting-all-vertex-lat-long-coordinates-every-1-meter-between-two-known-points
refering to http://www.movable-type.co.uk/scripts/latlong.html which has formulas in Java Script and for Excel

Python function adaptations


#### Importing all needed modules and libraries

In [5]:
import pandas as pd
import math

#### GIS functions needed for coordinates calculation

In [6]:
def getPathLength(lat1,lng1,lat2,lng2):
    '''calculates the distance between two lat, long coordinate pairs'''
    R = 6371000 # radius of earth in m
    lat1rads = math.radians(lat1)
    lat2rads = math.radians(lat2)
    deltaLat = math.radians((lat2-lat1))
    deltaLng = math.radians((lng2-lng1))
    a = math.sin(deltaLat/2) * math.sin(deltaLat/2) + math.cos(lat1rads) * math.cos(lat2rads) * math.sin(deltaLng/2) * math.sin(deltaLng/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d

def calculateBearing(lat1,lng1,lat2,lng2):
    '''calculates the azimuth in degrees from start point to end point'''
    startLat = math.radians(lat1)
    startLong = math.radians(lng1)
    endLat = math.radians(lat2)
    endLong = math.radians(lng2)
    dLong = endLong - startLong
    dPhi = math.log(math.tan(endLat/2.0+math.pi/4.0)/math.tan(startLat/2.0+math.pi/4.0))
    if abs(dLong) > math.pi:
         if dLong > 0.0:
             dLong = -(2.0 * math.pi - dLong)
         else:
             dLong = (2.0 * math.pi + dLong)
    bearing = (math.degrees(math.atan2(dLong, dPhi)) + 360.0) % 360.0;
    return bearing

def getDestinationLatLong(lat,lng,azimuth,distance):
    '''returns the lat and long of destination point 
    given the start lat, long, aziuth, and distance'''
    R = 6378.1 #Radius of the Earth in km
    brng = math.radians(azimuth) #Bearing is degrees converted to radians.
    d = distance/1000 #Distance m converted to km
    lat1 = math.radians(lat) #Current dd lat point converted to radians
    lon1 = math.radians(lng) #Current dd long point converted to radians
    lat2 = math.asin(math.sin(lat1) * math.cos(d/R) + math.cos(lat1)* math.sin(d/R)* math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng) * math.sin(d/R)* math.cos(lat1), math.cos(d/R)- math.sin(lat1)* math.sin(lat2))
    #convert back to degrees
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    return[lat2, lon2]


#### Load GIS and Apparent Resistivity data for HEP from CSV file

In [88]:
# load CSV in dictionary and df
# RESa data df
df_res = pd.read_csv('hep_4a.csv', skiprows=3)
#df_res = pd.read_csv('hep_5.csv', skiprows=6)
#df_res = pd.read_csv('hep_2.csv', skiprows=6)
df_res.dropna(axis=1, how='all', inplace=True)
df_res.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   148.6981132  78 non-null     float64
dtypes: float64(1)
memory usage: 752.0 bytes


In [89]:
print('count ' + df_res.count().astype(str))
df_res.shape[0]
#print('number of rows ' + len(df_res.index).astype(str))

148.6981132    count 78
dtype: object


78

In [77]:
# calculate K - Wenner
mn = 0.30
Kgeom = 2 * math.pi * mn
Kgeom

1.8849555921538759

In [90]:
# load start and finish points of HEP profile to a separate df
df_gis = pd.read_csv('hep_4a.csv', nrows=3, header=None)
df_gis

,0,1,2
0,Начало,56.038640,37.165744
1,Конец,56.039234,37.166315
2,шаг,1.000000,NaN


In [91]:
# prepare GIS data to calculate profile locations
lat1 = df_gis.iloc[0, 1]
lng1 = df_gis.iloc[0, 2]

lat2 = df_gis.iloc[1, 1]
lng2 = df_gis.iloc[1, 2]


In [92]:
# calculating length of the whole profile
HEP_dist = getPathLength(lat1,lng1,lat2,lng2)
HEP_dist

74.97066453126777

In [93]:
# calculate actual step of measurements - HEP_dist / count of df_res
HEP_step = HEP_dist / (df_res.shape[0] - 1)
HEP_step

0.9736449939125684

In [94]:
# calculate direction (azimuth or bearing of the profile)
HEP_dir = calculateBearing(lat1,lng1,lat2,lng2)
HEP_dir

28.235751662836492

In [80]:
def coord(interval,azimuth,lat1,lng1,lat2,lng2):
    '''returns every coordinate pair between two coordinate 
    pairs given the desired interval'''

    d = getPathLength(lat1,lng1,lat2,lng2)
    remainder, dist = math.modf((d / interval))
    counter = float(interval)
    coords = []
    coords.append([lat1,lng1])
    for distance in range(1,int(dist)):       # have to modify from original as it did not take into account the beginning/end point coordinates
        coord = getDestinationLatLong(lat1,lng1,azimuth,counter)
        counter = counter + float(interval)
        coords.append(coord)
    coords.append([lat2,lng2])
    return coords

In [95]:
# calculating lat-long for all RESa measurements
coord_list = coord(HEP_step,HEP_dir,lat1,lng1,lat2,lng2)
coord_list

[[56.03864, 37.165744],
 [56.03864770569808, 37.16575140727397],
 [56.03865541139571, 37.165758814550905],
 [56.03866311709289, 37.1657662218308],
 [56.03867082278962, 37.165773629113644],
 [56.038678528485924, 37.16578103639946],
 [56.038686234181775, 37.16578844368822],
 [56.038693939877184, 37.16579585097994],
 [56.038701645572154, 37.16580325827463],
 [56.038709351266675, 37.16581066557226],
 [56.038717056960756, 37.16581807287286],
 [56.03872476265438, 37.16582548017642],
 [56.038732468347575, 37.16583288748293],
 [56.038740174040335, 37.1658402947924],
 [56.038747879732625, 37.16584770210484],
 [56.03875558542448, 37.16585510942023],
 [56.03876329111591, 37.16586251673858],
 [56.03877099680689, 37.165869924059876],
 [56.0387787024974, 37.16587733138414],
 [56.038786408187484, 37.16588473871136],
 [56.03879411387713, 37.16589214604154],
 [56.03880181956633, 37.16589955337468],
 [56.03880952525509, 37.165906960710785],
 [56.0388172309434, 37.16591436804983],
 [56.038824936631265, 3

In [96]:
# converting coordinate list to df and merging with RESa data 
df_ptgis = pd.DataFrame(coord_list)
df_ptgis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       78 non-null     float64
 1   1       78 non-null     float64
dtypes: float64(2)
memory usage: 1.3 KB


In [97]:
# merge two df and add headers
df_erGIS = df_ptgis.join(df_res)
df_erGIS.columns = ['Latitude', 'Longitude', 'RESapparent']
df_erGIS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Latitude     78 non-null     float64
 1   Longitude    78 non-null     float64
 2   RESapparent  78 non-null     float64
dtypes: float64(3)
memory usage: 2.0 KB


In [98]:
# add REScorrect
df_erGIS['REScorrect'] = df_erGIS['RESapparent'] * Kgeom / 2.13

In [99]:
# calculating summary statistics
df_erGIS['RESapparent'].describe().round(1)

count     78.0
mean     166.5
std       43.2
min       88.2
25%      129.2
50%      163.6
75%      200.8
max      254.1
Name: RESapparent, dtype: float64

In [100]:
df_erGIS['REScorrect'].describe().round(1)

count     78.0
mean     147.3
std       38.2
min       78.1
25%      114.3
50%      144.8
75%      177.7
max      224.9
Name: REScorrect, dtype: float64

In [101]:
# saving results as CSV

df_erGIS.to_csv('gis_hep_4a.csv')
